# Numerical differentiation

This section focuses on ways to numerically approximate the derivative of a function $f(x)$ with respect to an independent variable $x$. If we know $f$ explicitly we can use symbolic manipulations find $f'=\frac{\text{d}f}{\text{d}x}$. Let's say that instead we only have data $f(x_n)$ sampled at $N$ points $x_0,x_1,...,x_n,...,x_{N-1}$. Taylor expanding $f$ at a point $x_n+h$ close to a given node $x_n$,

\begin{equation}
    f(x_n+h)=f(x_n) + hf'(x_n) + \frac{1}{2}h^2f''(x_n) + \mathcal{O}(h^3)
\end{equation}


Taking $h$ to be the separation between successive grid points, ``finite difference'' methods use taylor expansions to approximate derivatives with an error that scales with some power $h$. For example, the first order forward finite difference approximation writes

\begin{equation}
    f'(x_n)=\frac{1}{h}[f(x_n+h)-f(x_n)] + \mathcal{O}(h).
\end{equation}

Similarly, we might write

\begin{equation}
    f'(x_n)=\frac{1}{h}[f(x_n)-f(x_n-h)] + \mathcal{O}(h).
\end{equation}

Unfortunately, both forward and backward finite differences perform poorly. A better approach is to combine the Taylor expansions for $f(x_n+h)$ and $f(x_n-h)$ to produce the ``centred'' finite difference approximation

\begin{equation}
    f'(x_n)=\frac{1}{2h}[f(x_n+h)-f(x_n-h)] +  \mathcal{O}(h^2).
\end{equation}

Of course we can't apply this formula to the endpoints $x_0$ and $x_{N-1}$ of the grid, since there we don't have $x_n-h$ and $x_n+h$ (respectively). The following function applies this centred differences approximation on interior points, and forward/backward approximations on the endpoints:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def fundiff(ff,xx):
    '''
        computes the second-order finite differences approximation to the derivative of a 
        function f with values sampled at a grid-points xx, assuming that the grid-points
        in xx are evenly spaced
    '''
    # compute array of grid-spacing h, and check that xx are all evenly spaced:
    hh = xx[1] - xx[0]
    if np.any(abs(np.diff(xx) - hh)>1e-14):
        raise ValueError('expected evenly spaced grid')
    
    # initialize array to hold derivative values:
    dfdx = np.zeros_like(ff)
    
    # use centred differences to compute derivatives at interior points:
    dfdx[1:-1] = (ff[2:] - ff[:-2])/hh/2.
    
    # apply forward/backward formulae at endpoints:
    dfdx[0] = (ff[1] - ff[0])/hh
    dfdx[-1]= (ff[-1]- ff[-2])/hh
    return dfdx

Because this function applies a less accurate finite difference approximation at the boundaries, we expect the error to be larger there. The numpy function np.gradient uses much the same approach as above, but with some more bells and whistles (differentiation along one axis of an array, higher order approximations at boundaries):

In [ ]:
xx = np.linspace(0,1,100)
ff = np.cos(20*xx)*np.exp(-2*xx)
dfdx = -20*np.sin(20*xx)*np.exp(-2*xx) - 2*np.cos(20*xx)*np.exp(-2*xx)

plt.figure(figsize=(20,6))
plt.plot(xx,fundiff(ff,xx),label='fundiff',lw=5)
plt.plot(xx,np.gradient(ff,xx),label='np.gradient',lw=5)
plt.plot(xx,np.gradient(ff,xx,edge_order=2),label='np.gradient (2nd order edges)',lw=5)
plt.plot(xx,dfdx,label='truth',lw=5)
plt.xlabel('$x$',fontsize=16)
plt.ylabel("$f'(x)$",fontsize=16)
plt.legend(fontsize=14)

## Exercise

The second-order, forward, centred, and backward finite difference approximations for the \emph{second} derivatives of a function are
\begin{align}
    f''(x)
    &\simeq\frac{1}{h^2}[f(x + 2h) -2 f(x+h) + f(x)] \hspace{2em} \text{(forward)},
\\
    &\simeq\frac{1}{h^2}[f(x + h) -2f(x) + f(x - h)] \hspace{2em} \text{(centred)},
\\
    &\simeq\frac{1}{h^2}[f(x) - 2 f(x -h) + f(x - 2h)] \hspace{2em} \text{(backward)}.
\end{align}
Write a function that computes the second derivative of a function sampled at grid-points with a user-supplied constant separation h. Test it on the function considered above, and compare with the analytical solution.